In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fetch the webpage
url = "https://leagueoflegends.fandom.com/wiki/List_of_champions/Base_statistics"
response = requests.get(url)

if response.status_code == 200:
    page_content = response.text
else:
    print("Failed to fetch the webpage.")

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(page_content, "html.parser")

# Find the table based on its attributes
table = soup.find("table", {"class": "wikitable"})

# Extract table data into a Pandas DataFrame
table_data = []

for row in table.find_all("tr"):
    row_data = [cell.get_text(strip=True) for cell in row.find_all(["th", "td"])]
    table_data.append(row_data)

# Create a DataFrame from the table data
df = pd.DataFrame(table_data[1:], columns=table_data[0])

for c in df.columns:
        df = df.rename(columns={c: c.lower()})

for c in df.columns:
    if '5' in c:
        df = df.rename(columns={c: c.replace('5', '_regen')})

for c in df.columns:
    if '+' in c:
        df = df.rename(columns={c: c.replace('+', '_growth')})

In [3]:
df

,champions,hp,hp_growth,hp_regen,hp_regen_growth,mp,mp_growth,mp_regen,mp_regen_growth,ad,ad_growth,as,as_growth,ar,ar_growth,mr,mr_growth,ms,range
0,Aatrox,650,+114,3,+1,0,+0,0,+0,60,+5,0.651,+2.5%,38,+4.45,32,+2.05,345,175
1,Ahri,590,+96,2.5,+0.6,418,+25,8,+0.8,53,+3,0.668,+2%,21,+4.7,30,+1.3,330,550
2,Akali,570,+119,9,+0.9,200,+0,50,+0,62,+3.3,0.625,+3.2%,23,+4.7,37,+2.05,345,125
3,Akshan,630,+107,3.75,+0.65,350,+40,8.2,+0.7,52,+3,0.638,+4%,26,+4.7,30,+1.3,330,500
4,Alistar,685,+120,8.5,+0.85,350,+40,8.5,+0.8,62,+3.75,0.625,+2.125%,47,+4.7,32,+2.05,330,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,Zeri,630,+110,3.25,+0.7,250,+45,6,+0.8,53,+1.3,0.658,+2%,24,+4.2,30,+1.3,335,500
162,Ziggs,606,+106,6.5,+0.6,480,+23.5,8,+0.8,54,+3.1,0.656,+2%,22,+4.5,30,+1.3,325,550
163,Zilean,574,+96,5.5,+0.5,452,+50,11.35,+0.8,52,+3,0.625,+2.13%,24,+5,30,+1.3,335,550
164,Zoe,630,+106,7.5,+0.6,425,+25,8,+0.65,58,+3.3,0.625,+2.5%,21,+4.7,30,+1.3,340,550


In [4]:
for c in df.columns:
    df[c] = [r.replace('+', '') for r in df[c]] # remove +

df['as_growth'] = [float(r.replace('%', '')) / 100.0 for r in df['as_growth']]

df

,champions,hp,hp_growth,hp_regen,hp_regen_growth,mp,mp_growth,mp_regen,mp_regen_growth,ad,ad_growth,as,as_growth,ar,ar_growth,mr,mr_growth,ms,range
0,Aatrox,650,114,3,1,0,0,0,0,60,5,0.651,0.02500,38,4.45,32,2.05,345,175
1,Ahri,590,96,2.5,0.6,418,25,8,0.8,53,3,0.668,0.02000,21,4.7,30,1.3,330,550
2,Akali,570,119,9,0.9,200,0,50,0,62,3.3,0.625,0.03200,23,4.7,37,2.05,345,125
3,Akshan,630,107,3.75,0.65,350,40,8.2,0.7,52,3,0.638,0.04000,26,4.7,30,1.3,330,500
4,Alistar,685,120,8.5,0.85,350,40,8.5,0.8,62,3.75,0.625,0.02125,47,4.7,32,2.05,330,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,Zeri,630,110,3.25,0.7,250,45,6,0.8,53,1.3,0.658,0.02000,24,4.2,30,1.3,335,500
162,Ziggs,606,106,6.5,0.6,480,23.5,8,0.8,54,3.1,0.656,0.02000,22,4.5,30,1.3,325,550
163,Zilean,574,96,5.5,0.5,452,50,11.35,0.8,52,3,0.625,0.02130,24,5,30,1.3,335,550
164,Zoe,630,106,7.5,0.6,425,25,8,0.65,58,3.3,0.625,0.02500,21,4.7,30,1.3,340,550


In [5]:
df[df['champions'] == 'Draven']

,champions,hp,hp_growth,hp_regen,hp_regen_growth,mp,mp_growth,mp_regen,mp_regen_growth,ad,ad_growth,as,as_growth,ar,ar_growth,mr,mr_growth,ms,range
25,Draven,675,104,3.75,0.7,361,39,8.05,0.65,62,3.6,0.679,0.027,29,4.5,30,1.3,330,550


In [6]:
def get_stat(champ, stat, level):
    if stat == 'as':
        asp = float(df[df['champions'] == champ][stat])
        pct = float(df[df['champions'] == champ][stat+'_growth'])
        return asp * ((1 + pct) ** (level-1))
    elif stat == 'range':
        return float(df[df['champions'] == champ][stat])
    else:
        return float(df[df['champions'] == champ][stat]) + (level-1) * float(df[df['champions'] == champ][stat+'_growth'])

In [7]:
adc_list = ['Aphelios', 'Ashe', 'Caitlyn', 'Draven', 'Ezreal', 'Jhin', "Kai'Sa", "Kog'Maw", 'Lucian', 'Miss Fortune', 'Samira', 'Zeri']

#df[df['champions'].isin(adc_list)]

for s in ['hp', 'ad', 'as', 'ar', 'mr', 'range']:
    for i in adc_list:
        stat=get_stat(champ=i, stat=s, level=2)
        print(f'{i} {s}: {stat}')
    print('\n')

Aphelios hp: 682.0
Ashe hp: 741.0
Caitlyn hp: 687.0
Draven hp: 779.0
Ezreal hp: 702.0
Jhin hp: 762.0
Kai'Sa hp: 742.0
Kog'Maw hp: 734.0
Lucian hp: 741.0
Miss Fortune hp: 743.0
Samira hp: 738.0
Zeri hp: 740.0


Aphelios ad: 58.0
Ashe ad: 61.95
Caitlyn ad: 63.8
Draven ad: 65.6
Ezreal ad: 64.5
Jhin ad: 63.7
Kai'Sa ad: 61.6
Kog'Maw ad: 64.1
Lucian ad: 62.9
Miss Fortune ad: 54.4
Samira ad: 60.3
Zeri ad: 54.3


Aphelios as: 0.6534399999999999
Ashe as: 0.6799114
Caitlyn as: 0.7082400000000001
Draven as: 0.697333
Ezreal as: 0.640625
Jhin as: 0.64375
Kai'Sa as: 0.6555920000000001
Kog'Maw as: 0.6826225
Lucian as: 0.6590539999999999
Miss Fortune as: 0.6756800000000001
Samira as: 0.6797139999999999
Zeri as: 0.6711600000000001


Aphelios ar: 30.2
Ashe ar: 30.6
Caitlyn ar: 31.7
Draven ar: 33.5
Ezreal ar: 28.7
Jhin ar: 28.7
Kai'Sa ar: 32.2
Kog'Maw ar: 28.45
Lucian ar: 32.2
Miss Fortune ar: 32.2
Samira ar: 30.7
Zeri ar: 28.2


Aphelios mr: 31.3
Ashe mr: 31.3
Caitlyn mr: 31.3
Draven mr: 31.3
Ezreal mr:

Damage Dealt = Attack Damage×(1−(Armor/(Armor+100))

In League of Legends, it's important to note that the actual calculation of damage can be more complex due to factors like abilities, critical strikes, armor penetration, and other in-game mechanics. Different champions and abilities may have unique interactions, and some champions may deal different types of damage (e.g., magic damage or true damage) that are calculated differently.

Additionally, some abilities may apply damage over time (DoT) effects or additional damage based on the target's missing health, which adds further complexity to the calculation.

To calculate health lost from an attack in a specific in-game scenario, you would need to consider all relevant factors, such as champion abilities, items, and any buffs or debuffs, and apply the appropriate calculations accordingly. The exact details of these calculations can vary from champion to champion and may change with game updates, so it's important to refer to in-game tooltips, guides, or resources provided by the League of Legends community for precise calculations in specific situations.

In [14]:
def get_bonus_ad(item_ad):
    # adaptive force
    af = 5.4
    return af + item_ad

def get_q_ad(draven_level, item_ad=0):
    
    draven_bonus_ad = get_bonus_ad(item_ad)
    
    '''
    q_level_switch is a dictionary to identify how many skill points are in Draven's Q.
        key: maximum Draven level for the associated skill point value
        value: skill point value
    '''
    q_level_switch = {
        3: 1,
        4: 2,
        6: 3,
        7: 4,
        8: 5
    }

    found_q_level = False
    for i in q_level_switch.keys():
        if draven_level <= i and not found_q_level:
            q_level = q_level_switch[i]
            found_q_level = True
    
    '''
    q_ad_switch is a dictionary that contains Q's base AD and bonus damage percentage at each associated skill point value
        key: skill point value
        value: associated set of base AD and bonus damage percentage
    '''
    q_ad_switch = {
    1: [40, 0.75],
    2: [45, 0.85],
    3: [50, 0.95],
    4: [55, 1.05],
    5: [60, 1.15]
    }
    
    draven_base_ad = get_stat(champ='Draven', stat='ad', level=draven_level)
    q_base = q_ad_switch[q_level][0]
    q_bns_pct = q_ad_switch[q_level][1]
    
    return draven_base_ad + draven_bonus_ad + q_base + draven_bonus_ad * q_bns_pct

In [9]:
opp = 'Caitlyn'
draven_ad = get_stat(champ='Draven', stat='ad', level=2)


opp_hp = get_stat(champ=opp, stat='hp', level=2)
opp_ar = get_stat(champ=opp, stat='ar', level=2)


print(draven_ad, opp_hp, opp_ar)
draven_ad * (1 - (opp_ar / (opp_ar + 100)))

65.6 687.0 31.7


49.810174639331805

In [17]:
get_q_ad(1)

111.45